# Content Mining Models

## Table of Contents
##### [1. Data Preprocessing and Imports](#preprocessing)
##### [2. Models](#models)
###### [2.1. Cosine Similarity](#cosine)
###### [2.2. LSI Model](#lsi)
###### [2.3. Mixture Model](#mixture)
##### [3. Interpretation and Evaluation](#interpretation_evaluation)

<a id='preprocessing'></a>
## 1. Data Preprocessing and Imports

In [5]:
# Import functions module
%run functions.py

ModuleNotFoundError: No module named 'selenium'

<a id='models'></a>
## 2. Models

<a id='cosine'></a>
### 2.1. Cosine Similarity

<a id='lsi'></a>
### 2.2. LSI Model

<a id='mixture'></a>
### 2.3. Mixture Model

<a id='interpretation_evaluation'></a>
## 3. Interpretation and Evaluation